In [1]:
!pip install tensorflow


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns =['EIN','NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_type_val_count = application_df['APPLICATION_TYPE'].value_counts()
app_type_val_count

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_val_count[8:].index)
cutoff_value = 528

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_types = application_df['CLASSIFICATION'].value_counts()
classification_types

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classifications_cleaned = classification_types.loc[classification_types> 1]
classifications_cleaned

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_types.iloc[5:].index)
cutoff_value_classification = 1883

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
# Check the dataframe
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [11]:

# Check the data types for each column
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [12]:
# categorical_list will contain the names of all columns in application_df that have the data type "object", indicating they are categorical variables
categorical_list = application_df.dtypes[application_df.dtypes =="object"].index.tolist()
categorical_list

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
df_dummies = pd.get_dummies(application_df)
df_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [14]:
# The code extracts the values of the column named 'IS_SUCCESSFUL' from the DataFrame application_df.
# Specifically, it retrieves the values in that column and returns them as a NumPy array.

application_df['IS_SUCCESSFUL'].values

array([1, 1, 0, ..., 0, 1, 0], dtype=int64)

In [15]:
# Dropping IS_SUCCESSFUL column from application_df dataframe
df_new_application= application_df.drop(columns='IS_SUCCESSFUL')
df_new_application.values

array([['T10', 'Independent', 'C1000', ..., '0', 'N', 5000],
       ['T3', 'Independent', 'C2000', ..., '1-9999', 'N', 108590],
       ['T5', 'CompanySponsored', 'C3000', ..., '0', 'N', 5000],
       ...,
       ['T3', 'CompanySponsored', 'C2000', ..., '0', 'N', 5000],
       ['T5', 'Independent', 'C3000', ..., '0', 'N', 5000],
       ['T3', 'Independent', 'C1000', ..., '1M-5M', 'N', 36500179]],
      dtype=object)

In [16]:
# Check to see the array value in df_dummies dataframe
df_dummies['IS_SUCCESSFUL'].values



array([1, 1, 0, ..., 0, 1, 0], dtype=int64)

In [17]:
# Drop the column IS_SUCCESSFUL from the application_df dummy dataframe
df_clean_dummies = df_dummies.drop(columns='IS_SUCCESSFUL').values
df_clean_dummies

array([[1, 5000, False, ..., False, True, False],
       [1, 108590, False, ..., False, True, False],
       [1, 5000, False, ..., False, True, False],
       ...,
       [1, 5000, False, ..., False, True, False],
       [1, 5000, False, ..., False, True, False],
       [1, 36500179, False, ..., False, True, False]], dtype=object)

In [18]:
# Split our preprocessed data into our features and target arrays
y= df_dummies['IS_SUCCESSFUL'].values
X= df_dummies.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# The code calculates the total number of input features in the training data (X_train),  which represents a single data point.
input_features_total = len(X_train[0])
input_features_total

43

In [21]:
# Attempt 1.
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_total = len(X_train[0])  # total number of features

hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features_total, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

C:\Users\owenp\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 8)                   │             352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              45 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 403 (1.57 KB)

 Trainable params: 403 (1.57 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics= ['accuracy'])

In [23]:
# The shape of a NumPy array represents its dimensions, the number of elements in the array.
print(y_train.shape)

(25724,)


In [24]:
# Train the model
fit_model =nn.fit(X_train_scaled,y_train,epochs =100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 712us/step - accuracy: 0.6431 - loss: 0.6597
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 694us/step - accuracy: 0.7217 - loss: 0.5932
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 687us/step - accuracy: 0.7323 - loss: 0.5724
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 689us/step - accuracy: 0.7265 - loss: 0.5739
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 694us/step - accuracy: 0.7282 - loss: 0.5712
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 706us/step - accuracy: 0.7237 - loss: 0.5741
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 694us/step - accuracy: 0.7258 - loss: 0.5712
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 696us/step - accuracy: 0.7324 - loss: 0.5647
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 684us/step - accuracy: 0.7316 - loss: 0.5647
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 707us/step - accuracy: 0.7344 - loss: 0.5620
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 702us/step - accuracy: 0.7315 - loss: 0.5657
Epoch 12/100
804/80

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 574us/step - accuracy: 0.7290 - loss: 0.5655
Loss: 0.5655421614646912, Accuracy: 0.7289795875549316


In [6]:
from tensorflow.keras.models import Sequential
nn = Sequential()

filepath = r"C:\Users\owenp\OneDrive\Desktop\Module 21\Module21\Module21-main\HS_files\AlphabetSoupCharity.h5"

nn.save(filepath)

In [20]:
# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics= ['accuracy'])